In [1]:
%load_ext rpy2.ipython

/home/phoenix/miniconda3/envs/correlation/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [343]:
%%R

# The following function computes the new correlation coefficient, and returns the P-value for testing independence unless otherwise specified. Removes NAs and converts factor variables to integers automatically, unless otherwise specified. In general, it is safe to just supply x and y, and leave the other parameters to their default values.

# The following function computes the new correlation coefficient, and returns the P-value for testing independence unless otherwise specified. Removes NAs and converts factor variables to integers automatically, unless otherwise specified. In general, it is safe to just supply x and y, and leave the other parameters to their default values.

xi = function(x, y, pvalue = T, ties = T, method = "asymptotic", nperm = 1000, factor = T) {
    
    # x and y are the data vectors
    # The P-value for the test of independence is returned if pvalue = T. Otherwise, only the coefficient is returned. 
    # If ties = T, the algorithm assumes that the data has ties and employs the more elaborated theory for calculating the P-value. Otherwise, it uses the simpler theory. There is no harm in putting ties = T even if there are no ties.
    # method = "asymptotic" returns P-values computed by the asymptotic theory. If method = "permutation", a permutation test with nperm permutations is employed to estimate the P-value. Usually, there is no need for the permutation test. The asymptotic theory is good enough.
    # nperm is the number of permutations for the permutation test, if needed.
    # factor = T results in the algorithm checking whether x and y are factor variables and converting them to integers if they are. If it is known that the variables are numeric, a little bit of time can be saved by setting factor = F.
    
    
    # NAs are removed here:
    ok = complete.cases(x,y)
    x = x[ok]
    y = y[ok]    
    
    # Factor variables are converted to integers here:
    if (factor == T) {
        if (!is.numeric(x)) x = as.numeric(factor(x))
        if (!is.numeric(y)) y = as.numeric(factor(y))
    }
    
    
    n = length(x)                             # n is the sample size.
    
    PI = rank(x, ties.method = "random")        # PI is the rank vector for x, with ties broken at random 
    
    
    f = rank(y, ties.method = "max")/n        # f[i] is number of j s.t. y[j] <= y[i], divided by n.
    
    g = rank(-y, ties.method = "max")/n        # g[i] is number of j s.t. y[j] >= y[i], divided by n.
    
    ord = order(PI)                            # order of the x's, ties broken at random.
    
    f = f[ord]                                # Rearrange f according to ord.
    
    # xi is calculated in the next three lines:
    A1 = mean(abs(f[1:(n-1)] - f[2:n]))*(n-1)/(2*n)
    C = mean(g*(1-g))
    xi = 1 - A1/C
    
    
    # If P-value needs to be computed:
    if (pvalue == T) {
        
        # If there are no ties, return xi and theoretical P-value:
        if (ties == F) return(list(xi = xi, pval = 1 - pnorm(sqrt(n)*xi/sqrt(2/5))))
        
        # If there are ties, and the theoretical method is to be used for calculation P-values:
        if (method == "asymptotic") {
            
            # The following steps calculate the theoretical variance in the presence of ties:
            q = sort(f)
            ind = c(1:n)
            ind2 = 2*n - 2*ind + 1
            a = mean(ind2*q*q)/n
            c = mean(ind2*q)/n
            cq = cumsum(q)
            m = (cq + (n - ind)*q)/n
            b = mean(m^2)
            v = (a - 2*b + c^2)/(C^2)
            
            # Return xi and P-value:
            return(list(xi = xi, pval = 1 - pnorm(sqrt(n)*xi/sqrt(v))))
        }
        
        # If permutation test is to be used for calculating P-value:
        if (method == "permutation") {
            r = rep(0, nperm)
            for (i in 1:nperm) {
                x1 = runif(n, 0, 1)
                r[i] = xi(x1,y)$xi
            }
        
            # Return xi and P-value based on permutation test:
            return(list(xi = xi, pval = mean(r > xi)))
        }
        cat("Invalid method. Use either asymptotic or permutation.")
    }
    
    # If only xi is desired, return xi:
    else return(xi)
}



In [406]:
import pandas as pd
import numpy as np
import scipy.stats as ss
from scipy.stats import norm
from collections import namedtuple


class xi2:
    
    def __init__(
        self, 
        x, 
        y,
    ):
        self.raw_x = pd.Series(x)
        self.raw_y = pd.Series(y)
    
    @property
    def sample_size(self):
        return len(self.raw_x)
    
    @property 
    def data(self):
        CorrData = namedtuple('CorrData', ['x', 'y'])
        both_not_null = self.raw_x.isnull() & self.raw_y.isnull()
        x = self.raw_x[~both_not_null]
        y = self.raw_y[~both_not_null]  
        return CorrData(x, y)
    
    @property
    def x_ordered_rank(self):
        # PI is the rank vector for x, with ties broken at ordinal (needs to be random)
        return self.data.x.rank(method='dense')
        
        
    @property
    def y_rank_max(self):
        # f[i] is number of j s.t. y[j] <= y[i], divided by n.
        return self.data.y.rank(method="max")/n 
        
    @property
    def g(self):
        # g[i] is number of j s.t. y[j] >= y[i], divided by n.
        return pd.Series([1-i for i in self.data.y]).rank(method="max")/n 
    
    @property
    def x_ordered(self):
        # order of the x's, ties broken at random.
        return np.argsort(self.x_ordered_rank) 
    
    @property
    def x_rank_max_ordered(self):
        # Rearrange f according to ord.
        return [self.y_rank_max[i] for i in self.x_ordered]  

    @property
    def mean_absolute(self):
        return np.mean(
            np.abs(
                [
                    x-y for x,y 
                    in zip(
                        self.x_rank_max_ordered[0:(self.sample_size-1)], 
                        self.x_rank_max_ordered[1:self.sample_size]
                    )
                ]
            )
        )*(self.sample_size-1)/(2*self.sample_size) 
    
    @property
    def inverse_g_mean(self):
        # C = np.mean(self.g*(1-self.g))
        return np.mean(self.g*(1-self.g))
    
    @property
    def correlation(self):
        """xi correlation"""
        return 1 - self.mean_absolute/self.inverse_g_mean
    
    
class xiPValue:
    
    def __init__(self, xicorr):
        self.sample_size = xicorr.sample_size # xi correlation object for P val
        self.data = xicorr.data
        self.correlation = xicorr.correlation
        self.inverse_g_mean = xicorr.inverse_g_mean
        self.x_rank_max_ordered = xicorr.x_rank_max_ordered

    def asymptotic(self, ties=False, nperm=1000, factor=True):
        # If there are no ties, return xi and theoretical P-value:

        if ties:
            return 1-ss.norm.cdf(np.sqrt(self.sample_size)*self.correlation/np.sqrt(2/5))

        # If there are ties, and the theoretical method is to be used for calculation P-values:
        # The following steps calculate the theoretical variance in the presence of ties:
        q = sorted(self.x_rank_max_ordered)
        ind = [i+1 for i in range(self.sample_size)]
        ind2 = [2*self.sample_size - 2*ind[i-1]+1 for i in ind]
        a = np.mean([i*j*j for i,j in zip(ind2,q)])/self.sample_size
        c = np.mean([i*j for i,j in zip(ind2,q)])/self.sample_size
        cq = np.cumsum(q)
        m =[(i + (self.sample_size - j)*k)/n for i,j,k in zip(cq,ind,q)]
        b = np.mean([np.square(i) for i in m])
        v = (a - 2*b + np.square(c))/np.square(self.inverse_g_mean)
        return 1-ss.norm.cdf(np.sqrt(self.sample_size)*self.correlation/np.sqrt(v))
    
    def permutation_test(self, nperm=1000):
        # If permutation test is to be used for calculating P-value:
        r = np.zeros((nperm,), dtype=int)
        for idx,val in enumerate(r):
        # x1 = runif(n, 0, 1)
            x1 = np.random.uniform(self.sample_size, 0, 1)
            r[idx] = xi2(x1,self.data.y).correlation

        # Return xi and P-value based on permutation test:
        return (np.mean([ri for ri in r if ri > self.correlation]))
        

In [399]:
def xi(x, y):
    x= pd.Series(x)
    y= pd.Series(y)
    
    # remove NAs
    both_not_null = x.isnull() & y.isnull()
    x = x[~both_not_null]
    y = y[~both_not_null]
    
    # sample size
    n = len(x) 
    # PI is the rank vector for x, with ties broken at ordinal (needs to be random)
    PI = x.rank(method='dense')
    # f[i] is number of j s.t. y[j] <= y[i], divided by n.
    f = y.rank(method="max")/n        
    # g[i] is number of j s.t. y[j] >= y[i], divided by n.
    g = pd.Series([1-i for i in y]).rank(method="max")/n 
    # order of the x's, ties broken at random.
    ord = np.argsort(PI)   
    # Rearrange f according to ord.
    f = [f[i] for i in ord]                              

    A1 = np.mean(np.abs([x-y for x,y in zip(f[0:(n-1)], f[1:n])]))*(n-1)/(2*n)
    C = np.mean(g*(1-g))
    xi_val = 1 - A1/C
    return xi_val, n, f, C


def pvalue_asymptotic(xi_val, n, f, C, ties = False, nperm = 1000, factor = True):
    
    # If there are no ties, return xi and theoretical P-value:
    if ties:
        return 1-ss.norm.cdf(np.sqrt(n)*xi_val/np.sqrt(2/5))

    # If there are ties, and the theoretical method is to be used for calculation P-values:
    # The following steps calculate the theoretical variance in the presence of ties:
    q = sorted(f)
    ind = [i+1 for i in range(n)]
    ind2 = [2*n - 2*ind[i-1]+1 for i in ind]
    a = np.mean([i*j*j for i,j in zip(ind2,q)])/n
    c = np.mean([i*j for i,j in zip(ind2,q)])/n
    cq = np.cumsum(q)
    m =[(i + (n - j)*k)/n for i,j,k in zip(cq,ind,q)]
    b = np.mean([np.square(i) for i in m])
    v = (a - 2*b + np.square(c))/np.square(C)
    return 1-ss.norm.cdf(np.sqrt(n)*xi_val/np.sqrt(v))
    

def pvalue_permutation_test(nperm, n, y):
#     # If permutation test is to be used for calculating P-value:
    r = np.zeros((nperm,), dtype=int)
    for idx,val in enumerate(nperm):
        # x1 = runif(n, 0, 1)
        x1 = np.random.uniform(n, 0, 1)
        r[idx] = xi(x1,y)

        # Return xi and P-value based on permutation test:
        return (np.mean([ri for ri in r if ri > xi]))

    
def wrapper(x, y, pvalue=True, ties=False, method="asymptotic", nperm=1000, factor=True):
    xi_val, n, f, C = xi(x,y)
    if pvalue:
        if method == "asymptotic":
            pvalue = pvalue_asymptotic(xi_val, n, f, C, ties=ties, nperm=nperm, factor=factor)
        elif method == "permutation":
            pvalue = pvalue_permutation_test(nperm, n, y)
        else:
            print(f"method: {method} currently not supported, please select asymptotic or permutation")
            import sys
            sys.exit()
            
        return (xi_val, pvalue)
    return xi
            

In [404]:
# def xi(x, y, pvalue=True, ties=True, method="asymptotic", nperm=1000, factor=True):
x_i = [10,  8, 13,  9, 11, 14,  6,  4, 12,  7,  5]
y_i = [8.04, 6.95, 7.58, 8.81, 8.33, 9.96, 7.24, 4.26, 10.84, 4.82, 5.68]
x_ii = [10,  8, 13,  9, 11, 14,  6,  4, 12,  7,  5]
y_ii = [9.14, 8.14, 8.74, 8.77, 9.26, 8.1,  6.13, 3.1,  9.13, 7.26, 4.74]
pvalue = True
ties = False
method = "asymptotic"
nperm = 1000
factor = True

xi_val, pvalue = wrapper(x_i, y_i, pvalue=True, ties=False, method="asymptotic", nperm=1000, factor=True)
print(xi_val, pvalue)
# xi_val, pvalue = wrapper(x_i, y_i, pvalue=True, ties=False, method="permutation", nperm=1000, factor=True)
# print(xi_val, pvalue)

0.2749999999999999 0.07841556446646347


In [405]:
# xi class test
xi_val = xi2(x_i, y_i)
#  pvalue=True, ties=False, method="asymptotic", nperm=1000, factor=True
print(xi_val.correlation)
xi_p = xiPValue(xi_val)
print(xi_p.asymptotic(ties=False, nperm=1000, factor=True))
# print(xi_p.permutation_test(nperm=1000))

0.2749999999999999
0.07841556446646347


In [347]:
%%R

x_i = c(10,  8, 13,  9, 11, 14,  6,  4, 12,  7,  5)
y_i = c( 8.04, 6.95, 7.58, 8.81, 8.33, 9.96, 7.24, 4.26, 10.84, 4.82, 5.68) 
x_ii = c(10,  8, 13,  9, 11, 14,  6,  4, 12,  7,  5)
y_ii = c(9.14, 8.14, 8.74, 8.77, 9.26, 8.1,  6.13, 3.1,  9.13, 7.26, 4.74)
x_iii = c(10,  8, 13,  9, 11, 14,  6,  4, 12,  7,  5)
y_iii = c( 7.46,  6.77, 12.74,  7.11,  7.81,  8.84,  6.08,  5.39,  8.15,  6.42,  5.73)
x_iv = c( 8,  8,  8,  8,  8,  8,  8, 19,  8,  8,  8)
y_iv = c( 6.58,  5.76,  7.71,  8.84,  8.47,  7.04,  5.25, 12.5 ,  5.56,  7.91,  6.89)

xi(x_i, y_i, pvalue = T, ties = T, method = "asymptotic", nperm = 1000, factor = T)


$xi
[1] 0.275

$pval
[1] 0.07841556

